# CTP using Gibb's sampling

LDA deel op basis van uitleg in http://u.cs.biu.ac.il/~89-680/darling-lda.pdf

Using Gensim functionality

Dus we mogen ervan uitgaan dat we een dictionary hebben en bow vectors voor de documenten.
Het een en ander aan counts komt dan uit gensim. Maar hoe?

Wat hebben we nodig (LDA deel)?

* The number of words assigned to topic k in document d: $$n_{d;k}$$
* The number of times word w is assigned to topic k: $$n_{k;w}$$
* The total number of times any word is assigned to topic k: $$n_k$$
* Array z which will contain the current topic assignment for each of the N words in the corpus.

We beginnen met 1 extra perspectief (misschien perspectief-klasse?)

In [1]:
# Generate documents
import numpy as np

length_topic = 50
length_opinion = 20
num_topics = 3
topic_vocabulary = np.array(['zon', 'ijs', 'strand', 'vanille', 'chocola', 'broccoli', 'wortel'])
opinion_vocabulary = np.array(['warm', 'zwemmen', 'zonnig', 'bewolkt', 'vies', 'lekker', 'koud'])

real_theta_topic = np.array([[1.0, 0.0, 0.0],
                             [0.0, 1.0, 0.0],
                             [0.0, 0.0, 1.0],
                             [0.5, 0.5, 0.0],
                             [0.0, 0.5, 0.5]])
real_phi_topic = np.array([[0.4, 0.2, 0.4, 0.0, 0.0, 0.0, 0.0],
                           [0.0, 0.3, 0.0, 0.35, 0.35, 0.0, 0.0],
                           [0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5]])
real_phi_opinion = np.array([[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                             [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
                             [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]])

In [2]:
# generate the corpus
from collections import Counter

corpus = []

for m, tm in enumerate(real_theta_topic):
    doc = {'topic': [], 'opinion': []}
    topic_counter = Counter()
    for i in range(length_topic):
        # topic words
        topic = np.random.multinomial(1, tm)
        topic_index = np.where(topic==1)[0][0]
        topic_counter[topic_index] += 1
        word = np.random.multinomial(1, real_phi_topic[topic_index])
        doc['topic'].append(topic_vocabulary[np.where(word==1)[0][0]])
    
    #print topic_counter
    
    # select opinion (index) based on topic occurrence
    om = np.array([float(topic_counter[i]) for i in range(num_topics)])
    #print om
    # normalize
    om /= sum(om)
    for i in range(length_opinion):
        # opinion words
        topic = np.random.multinomial(1, om)
        topic_index = np.where(topic==1)[0][0]
        #print topic_index, real_phi_opinion[topic_index]
        word = np.random.multinomial(1, real_phi_opinion[topic_index])
        #print opinion_vocabulary[np.where(word==1)[0][0]]
        doc['opinion'].append(opinion_vocabulary[np.where(word==1)[0][0]])
        
    corpus.append(doc)
    
for i, doc in enumerate(corpus):
    print 'document', i+1, doc

document 1 {'topic': ['zon', 'strand', 'zon', 'ijs', 'ijs', 'zon', 'zon', 'ijs', 'zon', 'strand', 'zon', 'zon', 'strand', 'strand', 'strand', 'zon', 'ijs', 'strand', 'ijs', 'strand', 'ijs', 'strand', 'strand', 'strand', 'strand', 'ijs', 'zon', 'zon', 'ijs', 'ijs', 'ijs', 'zon', 'ijs', 'strand', 'strand', 'zon', 'ijs', 'ijs', 'strand', 'ijs', 'strand', 'strand', 'zon', 'zon', 'strand', 'ijs', 'ijs', 'strand', 'strand', 'strand'], 'opinion': ['warm', 'warm', 'warm', 'warm', 'warm', 'warm', 'warm', 'warm', 'warm', 'warm', 'warm', 'warm', 'warm', 'warm', 'warm', 'warm', 'warm', 'warm', 'warm', 'warm']}
document 2 {'topic': ['chocola', 'vanille', 'vanille', 'chocola', 'ijs', 'ijs', 'vanille', 'vanille', 'vanille', 'vanille', 'chocola', 'ijs', 'ijs', 'vanille', 'vanille', 'ijs', 'ijs', 'vanille', 'ijs', 'vanille', 'chocola', 'chocola', 'ijs', 'vanille', 'chocola', 'vanille', 'chocola', 'chocola', 'chocola', 'chocola', 'chocola', 'chocola', 'chocola', 'chocola', 'chocola', 'chocola', 'vanille

In [3]:
from gensim import corpora

d = corpora.Dictionary(corpus)

In [4]:
# initialize z and o
import random
from collections import Counter

# topic
z = {}
ndk = np.zeros((len(corpus), num_topics), dtype=np.int)
nkw = np.zeros((num_topics, len(topic_vocabulary)), dtype=np.int)
nk = Counter()

# opinion
x = {}
nsd = np.zeros((len(corpus), num_topics), dtype=np.int)
nrs = np.zeros((num_topics, len(opinion_vocabulary)), dtype=np.int)
ntd = np.zeros(len(corpus), dtype=np.int)
ns = Counter()

for d, doc in enumerate(corpus):
    for i, word in enumerate(doc['topic']):
        t = random.randint(0,2)
        z[(d, i)] = t
        ndk[d][t] += 1
        word_index = np.where(topic_vocabulary==word)[0][0]
        nkw[t][word_index] += 1
        nk[t] += 1
        ntd[d] += 1
    for i, word in enumerate(doc['opinion']):
        o = random.randint(0,2)
        x[(d, i)] = o
        nsd[d][o] += 1
        word_index = np.where(opinion_vocabulary==word)[0][0]
        nrs[o][word_index] += 1
        ns[o] += 1

#print z
#print ndk
#print np.sum(ndk)
#print nkw
#print np.sum(nkw)
#print nk
print x
print nsd
print nrs
print ntd
print nk, ns

{(1, 17): 0, (0, 7): 0, (1, 6): 0, (0, 10): 1, (3, 7): 1, (2, 5): 1, (1, 11): 2, (4, 0): 1, (4, 19): 1, (0, 17): 2, (0, 4): 2, (1, 1): 1, (4, 10): 2, (3, 2): 0, (2, 6): 2, (4, 5): 0, (4, 16): 1, (2, 17): 1, (0, 1): 2, (3, 12): 1, (1, 12): 2, (4, 15): 2, (3, 1): 2, (2, 11): 2, (3, 18): 2, (2, 18): 0, (0, 14): 1, (3, 11): 2, (2, 1): 1, (1, 15): 1, (4, 12): 2, (2, 12): 0, (3, 17): 2, (1, 16): 0, (1, 5): 0, (0, 11): 0, (3, 6): 2, (2, 2): 0, (1, 10): 1, (4, 1): 2, (0, 18): 0, (1, 19): 1, (0, 5): 0, (1, 0): 0, (0, 8): 1, (4, 11): 0, (3, 5): 0, (2, 7): 0, (4, 6): 2, (4, 17): 0, (0, 2): 2, (3, 15): 2, (1, 3): 0, (4, 8): 1, (3, 0): 2, (2, 8): 1, (2, 19): 2, (0, 15): 0, (3, 10): 2, (1, 14): 1, (4, 13): 0, (2, 13): 1, (3, 16): 0, (1, 4): 0, (0, 12): 0, (3, 9): 0, (2, 3): 0, (1, 9): 0, (4, 2): 0, (2, 14): 1, (0, 19): 2, (1, 18): 2, (0, 6): 0, (1, 7): 2, (0, 9): 1, (3, 4): 2, (2, 4): 0, (4, 7): 0, (4, 18): 1, (0, 16): 0, (0, 3): 1, (3, 14): 2, (1, 2): 0, (4, 9): 0, (3, 3): 0, (2, 9): 1, (4, 4): 0, 

In [5]:
# iterate
#np.seterr(all='raise')

def p_z(alpha, beta, num_topics, num_words, d, topic, word_index):
    #print alpha, beta, num_topics, num_words, d, topic, word_index
    #print ndk[d][topic], nkw[topic][word_index], nk[topic]
    
    # De formule voor p_z is anders in het uitgewerkte voorbeeld dan in paper 'Finding scientific topics' en in CTP paper 
    # die hier staat is die uit het uitgewerkte voorbeeld
    return (ndk[d][topic]+alpha)*(nkw[topic][word_index]+beta)/(nk[topic]+beta*num_words)

def p_z2(alpha, beta, num_topics, num_words, d, topic, word_index):
    return ((ndk[d][topic]+alpha)/(np.sum(ndk[d])+num_topics*alpha)) * \
           (nkw[topic][word_index]+beta)/(nk[topic]+beta*num_words)


def p_x(beta_o, num_opinion_words, d, opinion, word_index):
    #print beta_o, num_opinion_words, d, opinion, word_index
    #print nrs[opinion][word_index], nsd[d][opinion], ntd[d]
    left = (nrs[opinion][word_index]+beta_o)/ns[o]+beta_o*num_opinion_words
    right = (float(nsd[d][opinion])/ntd[d])
    return left*right

def normalize(p):
    minimum = np.min(p)
    maximum = np.max(p)
    
    if minimum < 0:
        print '<0'
        return normalize((p - minimum)/(maximum - minimum))
    return p/sum(p)

num_iter = 100

alpha = 0.1
beta = beta_o = 0.02

theta = np.zeros((num_iter,  len(corpus), num_topics), dtype=np.float_)
phi = np.zeros((num_iter, num_topics, len(topic_vocabulary)), dtype=np.float_)
phi_o =np.zeros((num_iter, num_topics, len(opinion_vocabulary)), dtype=np.float_) 

for t in range(num_iter):
    for d, doc in enumerate(corpus):
        # Topic words
        for i, w in enumerate(doc['topic']):
            word = (d, i)
            topic = z[word]
            
            word_index = np.where(topic_vocabulary==w)[0][0]
            
            #if ndk[d][topic] > 0 and nkw[topic][word_index] > 0 and nk[topic] > 0:
            if True:
                ndk[d][topic] -= 1
                nkw[topic][word_index] -= 1
                nk[topic] -= 1
            
                p = [p_z2(alpha, beta, len(real_phi_topic), len(topic_vocabulary), d, j, word_index) for j in range(num_topics)]
                #print p
                # normalize
                p = normalize(p)
                #print p, sum(p)
            
                to = np.random.multinomial(1, p)
                topic = np.where(to==1)[0][0]
                
                z[word] = topic
            
                ndk[d][topic] += 1
                nkw[topic][word_index] += 1
                nk[topic] += 1

        # Opinion words
        for i, w in enumerate(doc['opinion']):
            word = (d, i)
            opinion = x[word]
            
            word_index = np.where(opinion_vocabulary==w)[0][0]
            
            nsd[d][opinion] -= 1
            nrs[opinion][word_index] -= 1
            ns[opinion] -= 1
            
            p = np.array([p_x(beta_o, len(opinion_vocabulary), d, j, word_index) for j in range(num_topics)], dtype=np.float)
            #print p, 
            p = normalize(p)
            #print p
            opinion = np.random.multinomial(1, p).argmax()
            #opinion = np.where(op==1)[0][0]
            
            x[word] = opinion
            
            nsd[d][opinion] += 1
            nrs[opinion][word_index] += 1
            ns[opinion] += 1
        
    # calculate theta and phi
    theta[t] = (ndk+float(alpha))/(np.sum(ndk, axis=1, keepdims=True)+num_topics*alpha)
    phi[t] = (nkw+float(beta))/(np.sum(nkw, axis=1, keepdims=True)+len(topic_vocabulary)*beta)    
    phi_o[t] = (nrs+float(beta_o))/(np.sum(nrs, axis=1, keepdims=True)+len(opinion_vocabulary)*beta_o)
            #print 'new topic', topic
print ndk
print nkw
print nk
print nrs
print ns

[[ 0  0 50]
 [50  0  0]
 [ 0 50  0]
 [33  0 17]
 [20 30  0]]
[[ 0 36  0 35 32  0  0]
 [ 0  0  0  0  0 39 41]
 [21 19 27  0  0  0  0]]
Counter({0: 103, 1: 80, 2: 67})
[[26  0  0  0  0  0 14]
 [ 0  0  0  0  0  0  1]
 [ 0  0  0  0 33  0 26]]
Counter({2: 59, 0: 40, 1: 1})


/usr/local/lib/python2.7/dist-packages/IPython/kernel/__main__.py:19: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python2.7/dist-packages/IPython/kernel/__main__.py:21: RuntimeWarning: invalid value encountered in double_scalars


In [6]:
print 'theta'
print 'found'
print np.mean(theta, axis=0)
print 'real'
print real_theta_topic
print
print 'phi'
print 'found'
print np.mean(phi, axis=0)
print 'real'
print real_phi_topic
print
print 'topics found:'
indexes = np.mean(phi, axis=0) > 0.01
for index in indexes:
    print topic_vocabulary[index]
print
print 'topics real'
indexes = real_phi_topic > 0.01
for index in indexes:
    print topic_vocabulary[index]
print 'opinions found:'
indexes = np.mean(phi_o, axis=0) > 0.01
for index in indexes:
    print opinion_vocabulary[index]
print
print 'opinions real'
indexes = real_phi_opinion > 0.01
for index in indexes:
    print opinion_vocabulary[index]

theta
found
[[ 0.01073559  0.00437376  0.98489066]
 [ 0.98807157  0.00795229  0.00397614]
 [ 0.00357853  0.99304175  0.00337972]
 [ 0.54592445  0.00417495  0.4499006 ]
 [ 0.38866799  0.6         0.01133201]]
real
[[ 1.   0.   0. ]
 [ 0.   1.   0. ]
 [ 0.   0.   1. ]
 [ 0.5  0.5  0. ]
 [ 0.   0.5  0.5]]

phi
found
[[  1.84446523e-03   3.07741933e-01   2.90573614e-03   3.57081507e-01
    3.27949131e-01   6.34212374e-04   1.84301510e-03]
 [  4.42282903e-04   3.90244848e-03   6.36194091e-04   2.41400546e-03
    1.86671274e-03   4.83797569e-01   5.06940787e-01]
 [  2.87205292e-01   3.38384339e-01   3.68730261e-01   1.97782981e-03
    7.94017445e-04   9.23651876e-04   1.98460840e-03]]
real
[[ 0.4   0.2   0.4   0.    0.    0.    0.  ]
 [ 0.    0.3   0.    0.35  0.35  0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.5   0.5 ]]

topics found:
['ijs' 'vanille' 'chocola']
['broccoli' 'wortel']
['zon' 'ijs' 'strand']

topics real
['zon' 'ijs' 'strand']
['ijs' 'vanille' 'chocola']
['broccoli' 'wortel

In [7]:
def print_topic(weights, words):
    l = zip(words, weights)
    l.sort(key=lambda tup: tup[1])
    print l[::-1]

for t in np.mean(phi_o, axis=0):
    print_topic(t, opinion_vocabulary)

[('warm', 0.64248298080849353), ('koud', 0.35205398075466376), ('vies', 0.0034848311894939425), ('lekker', 0.00049455181183695517), ('bewolkt', 0.00049455181183695517), ('zonnig', 0.00049455181183695517), ('zwemmen', 0.00049455181183695517)]
[('koud', 0.88700271544967546), ('vies', 0.025787147613644135), ('warm', 0.018007798250136348), ('lekker', 0.017300584671635637), ('bewolkt', 0.017300584671635637), ('zonnig', 0.017300584671635637), ('zwemmen', 0.017300584671635637)]
[('vies', 0.5573144753999546), ('koud', 0.43980912627046043), ('warm', 0.0015055458262555261), ('lekker', 0.00034271312583213553), ('bewolkt', 0.00034271312583213553), ('zonnig', 0.00034271312583213553), ('zwemmen', 0.00034271312583213553)]


Is het toeval dat de indices van de topic topics en de opinion topics overeen komen of niet? En hoe kun je daarachter komen?